<a href="https://colab.research.google.com/github/DariusTheGeek/Flood-Prediction-in-Malawi--Zindi-Competition/blob/master/Malawi_Flood_Prediction__starter_code__by_DariusMoruri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the Necessary Libraries

In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np

## Reading the Data

In [2]:

sub = pd.read_csv('C:\Current_Work\FloodPrediction\SampleSubmission.csv')
df=pd.read_csv('C:\Current_Work\FloodPrediction\Train.csv')

## Data Cleaning

In [3]:
precip_features_2019 = []
precip_features_2015 = []
for col in df.columns:
    if '2019' in col:
        precip_features_2019.append(col)
    elif 'precip 2014' in col:
        precip_features_2015.append(col)
    elif 'precip 2015' in col:
        precip_features_2015.append(col)
train=tain = df[df.columns.difference(precip_features_2019)]
precip_features_2019.extend(['X',	'Y',	'elevation', 'LC_Type1_mode',	'Square_ID'])
test = df[precip_features_2019]
new_2015_cols = {}
for col, number in zip(precip_features_2015, range(1, len(precip_features_2015) + 1)):

    if 'precip' in col:
        new_2015_cols[col] = 'week_' + str(number) + '_precip'
new_2019_cols = {}
for col, number in zip(precip_features_2019, range(1, len(precip_features_2019) + 1)):
    if 'precip' in col:
        new_2019_cols[col] = 'week_' + str(number) + '_precip'
train.rename(columns = new_2015_cols, inplace = True)
test.rename(columns = new_2019_cols, inplace = True)
target = train.target_2015
train, test = train.align(test, join = 'inner', axis = 1)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
X = train.drop(['Square_ID'], axis = 1)
y = target

C:\Users\ORAEGBU AYOMIDE\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


### key point: The flood may occur in consequence of heavy precipitation that occur during a period less than 18 weeks. For that reason we have just worked on the period with a continuios heavy precip.


In [4]:
train[train.columns[5:-1]].describe().loc[['mean']]

,week_1_precip,week_2_precip,week_3_precip,week_4_precip,week_5_precip,week_6_precip,week_7_precip,week_8_precip,week_9_precip,week_10_precip,week_11_precip,week_12_precip,week_13_precip,week_14_precip,week_15_precip,week_16_precip
mean,1.61076,2.502058,1.162076,8.27061,8.892459,9.572821,22.925036,28.11321,58.859208,1.251173,34.653177,28.314888,12.487909,3.802584,17.072285,9.110949


### Max precip within 2 weeks

In [5]:
train['week_7_precip_']=train['week_7_precip']+train['week_6_precip']
test['week_7_precip_']=test['week_7_precip']+test['week_6_precip']
train['week_8_precip_']=train['week_8_precip']+train['week_7_precip']
test['week_8_precip_']=test['week_8_precip']+test['week_7_precip']
train['week_9_precip_']=train['week_9_precip']+train['week_8_precip']
test['week_9_precip_']=test['week_9_precip']+test['week_8_precip']
train['max_2_weeks']=train[['week_7_precip_','week_8_precip_','week_9_precip_']].apply(np.max,axis=1)
test['max_2_weeks']=test[['week_7_precip_','week_8_precip_','week_9_precip_']].apply(np.max,axis=1) 

In [6]:

X1=train[['LC_Type1_mode', 'X', 'Y', 'elevation','week_7_precip', 'week_8_precip', 'week_9_precip','max_2_weeks']]
sub1=test[['LC_Type1_mode', 'X', 'Y', 'elevation','week_7_precip', 'week_8_precip', 'week_9_precip','max_2_weeks']]
X1.columns=sub1.columns


### Index of the week within the max of precip

In [7]:
def index(col):
    l=list(col)
    return l.index(max(l))

X1['max_index']=train[['week_6_precip', 'week_7_precip', 'week_8_precip','week_9_precip']].apply(index,axis=1)
sub1['max_index']=test[['week_6_precip', 'week_7_precip', 'week_8_precip','week_9_precip']].apply(index,axis=1)



<ipython-input-7-0e8cd181197e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X1['max_index']=train[['week_6_precip', 'week_7_precip', 'week_8_precip','week_9_precip']].apply(index,axis=1)
<ipython-input-7-0e8cd181197e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub1['max_index']=test[['week_6_precip', 'week_7_precip', 'week_8_precip','week_9_precip']].apply(index,axis=1)


### Precip variation over different weeks

In [8]:
sub1['slope_8_7']=((test['week_8_precip']/test['week_7_precip'])>1)*1
X1['slope_8_7']=((train['week_8_precip']/train['week_7_precip'])>1)*1


sub1['slope_9_8']=((test['week_9_precip']/test['week_8_precip'])>1)*1
X1['slope_9_8']=((train['week_9_precip']/train['week_8_precip'])>1)*1


<ipython-input-8-738f3592f7c2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub1['slope_8_7']=((test['week_8_precip']/test['week_7_precip'])>1)*1
<ipython-input-8-738f3592f7c2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X1['slope_8_7']=((train['week_8_precip']/train['week_7_precip'])>1)*1
<ipython-input-8-738f3592f7c2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

### outlier detection

In [9]:
from sklearn.covariance import EllipticEnvelope
clf2 = EllipticEnvelope(contamination=.17,random_state=0)
clf2.fit(X1)
ee_scores = pd.Series(clf2.decision_function(X1))
clusters2 = clf2.predict(X1)
X1['target']=target
X1['pred']=clusters2
X1=X1[X1['pred']!=-1]
X1,y=X1.drop(columns=['pred','target']),X1['target']

C:\Users\ORAEGBU AYOMIDE\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\covariance\_robust_covariance.py:166: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-59.581115490381272 > -60.461679265486765). You may want to try with a higher value of support_fraction (current value: 0.503).
  warnings.warn("Determinant has increased; this should not happen: "
C:\Users\ORAEGBU AYOMIDE\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\covariance\_robust_covariance.py:166: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-60.601847897661685 > -61.571491156168200). You may want to try with a higher value of support_fraction (current value: 0.503).
  warnings.warn("Determinant has increased; this should not happen: "
C:\Users\ORAEGBU AYOMIDE\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\covariance\_robust_covariance.py:166: RuntimeWarning: Determinan

In [10]:
sub1=X1

In [11]:
def metric(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

### Training model

In [12]:
params = { 'learning_rate':0.07,'max_depth':8}
X=X1
X_test=sub1

n_estimators = 221

n_iters = 5
preds_buf = []
err_buf = []
for i in range(n_iters): 
    x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=i)
    d_train = lgb.Dataset(x_train, label=y_train)
    d_valid = lgb.Dataset(x_valid, label=y_valid)
    watchlist = [d_valid]

    model = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=1)

    preds = model.predict(x_valid)
   
 
    err_buf.append(metric(model.predict(x_valid),y_valid))
    
    
    preds = model.predict(X_test)
    
    preds_buf.append(preds)

print('Mean RMSLE = ' + str(np.mean(err_buf)) + ' +/- ' + str(np.std(err_buf)))
# Average predictions
preds1 = np.mean(preds_buf, axis=0)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000898 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1280
[LightGBM] [Info] Number of data points in the train set: 10932, number of used features: 9
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score 0.078343
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

In [ ]:
def check(col):
    if col<0:
        return 0
    elif col>1:
        return 1
    else:
        return col
preds=preds1
preds-=0.08
submission_df = pd.DataFrame({'Square_ID': test.Square_ID, 'target_2019': preds}) 
submission_df['target_2019']=submission_df['target_2019'].apply(check)
submission_df.to_csv('lili.csv', index = False)